In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import string
import re
import matplotlib.pyplot as plt
from src.sentimentAnalysis import detect_sentiment, setRating
import cPickle as pickle
from pymongo import MongoClient
from collections import defaultdict
import enchant
from nltk.corpus import stopwords
%matplotlib inline

In [2]:
connection = MongoClient()
db = connection.wta
tripReports = db.trip_reports

In [3]:
def mongo2PandasClean(mongodb, clean_creator=True, drop_id=True):
    df = pd.DataFrame(list(mongodb.find()))
    if drop_id:
        del df['_id']
    return df

In [4]:
df = mongo2PandasClean(tripReports)

In [6]:
df.shape

(77725, 7)

In [14]:
f = open('tr.txt', 'w')
for text in df['Text'][:1000]:
    try:
        f.write(text)
    except:
        continue

In [18]:
df.head(20)

,Creator,Date,Rating,Text,author_id,hike_id,hike_name
0,amart005,"Jul 31, 2016",2,"The first meadow is a marshy mess, but after t...",12474,0,Grand Park via Lake Eleanor
1,xandwich,"Jul 26, 2016",3,Well...we got to Mount Rainier Tuesday morning...,19583,0,Grand Park via Lake Eleanor
2,HeatherD,"Jul 25, 2016",1,Found hiking poles at Lake Eleanor. Comment wi...,4317,0,Grand Park via Lake Eleanor
3,Lindy99,"Jul 24, 2016",4,"The WTA guide says this hike is 1,100 foot ele...",6676,0,Grand Park via Lake Eleanor
4,SpartanHiker,"Jul 23, 2016",4,Wonderful hike to Grand park. Overall the tra...,10392,0,Grand Park via Lake Eleanor
5,hdpMusik3,"Jul 21, 2016",2,"This was a four-day, 40-mile backpack with fou...",14693,0,Grand Park via Lake Eleanor
6,DKreuger,"Jul 17, 2016",3,Hit the trailhead about 10:15 a.m. with a grou...,2501,0,Grand Park via Lake Eleanor
7,Cascadian Hiker,"Jul 16, 2016",2,"I hiked Grand Park last year and it was great,...",1962,0,Grand Park via Lake Eleanor
8,Alien Dave,"Jul 15, 2016",4,"(Revisited on Friday, July 15th - update for t...",495,0,Grand Park via Lake Eleanor
9,"[email protected]/* <![CDATA[ */!function(t,e,...","Jul 15, 2016",4,Nice hike after a few potholes on drive up for...,12230,0,Grand Park via Lake Eleanor


In [16]:
df['Text'][0]

u"The first meadow is a marshy mess, but after that the trail is in great shape. The bugs were awful along the trail but not bad once at Grand Park thanks to a cool breeze. Started at 8 am and came upon a mama bear and two cubs just off the trail about 100 yds past Lake Eleanor. Mama  and one cub ran right, other cub ran left of the trail- not ideal, so we backed down the trail a ways and kept her in our sights- she didn't appear agitated or frightened. Eventually the three got back together and headed deeper into the forest.  Saw them again on our descent, at the end of the first meadow. Mama was munching and cubs were playing. She watched us briefly but was undeterred. Beautiful to watch them. Good reminder to make noise along the trail- we got lazy about it as we were so close to the camp and could hear people talking. \rOh yeah- the mountain was out and the flowers were lovely. And also- bear cubs!!"

In [23]:
df['Text'][7]

u"I hiked Grand Park last year and it was great, but it was later in the year and pretty dry, so there weren't any wildflowers in evidence. I decided a few months ago that I wanted to go back when the meadow was blooming, which lead to this trip today.\r The road to the trail is in ok condition, but there are definitely plenty of potholes. You'll want to take things slowly, but overall you shouldn't have too much trouble. The roads are well-signed, but there's no sign for the trailhead itself. Instead look for the sign for Eleanor Creek and park near that.\r The trail has some really wet and muddy spots between the trailhead and the small meadow you come to after Lake Eleanor. You can try to pick your way through, stepping from partially submerged rock to slippery log, but unless you're prepared to bushwhack way off trail your feet are going to get messy. I wore my gaiters knowing that this was likely to be the case. There are also a fair number of blowdowns along the trail between the

In [22]:
df['Text'][15]

u"I'm usually bad about writing trip reports, but I just wanted to get on here to clarify a few things I saw in previous trip reports.\r The road in is long and gravel but in pretty good shape. There are a few potholes here and there but there is always a way around, and there were no rough sections or washboard. Once you make the turn from Hwy. 410 it's really hard to get lost as you basically follow the same road until you get to Eleanor Creek (the second creek crossing: Huckleberry Creek is the first). We went in the middle of the holiday weekend and there were quite a few cars there but it was by no means overflowing, so I have to think it would be rare that you'd have to park any appreciable distance from the trailhead.\r Trail conditions are great. There may have been a little snow a week ago but as of now there is none on the trail. At a couple points it's still close enough to make the trail damp and muddy, but that's it. There is also one big mud puddle very close to the entra

In [20]:
df['Text'][13]

u"This is a favorite hike, and even though the low clouds kept us from seeing the amazing Mt Rainier views it was still a wonderful day! \rPrevious trip reports said that the trail is muddy between the trail head and the lake.... Having hiked this before I didn't pay much attention because it's always a little muddy there... But it is REALLY muddy!! Passable, but very muddy. There are also many trees down on the trail. All can be easily climbed over, under or walked around. \rLots of wildflowers blooming; bear grass, Avalanche lilies, lupine, magenta paintbrush, yellow flowers whose name I don't know, and mountain Heather beginning to bloom too. \rA pair of hikers coming down the trail as we were headed up saw a bear in the first meadow. We saw bear scat more than once, but no bears. \rThe mosquitos were pretty thick before and around the lake, but not bad after that. \rA couple more days of sunshine and the flowers will really be impressive!"